In [1]:
# Session configuration with Spark on Yarn with shared modules built with conda
import os
import pyspark
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
spark_session = SparkSession.builder.master("yarn") \
                    .appName('spark-yarn-4vms') \
                    .config(
                        "spark.archives",  # 'spark.yarn.dist.archives' in YARN.
                        "hdfs://192.168.2.250:9000/user/ubuntu/share/envs/pyspark_conda_env.tar.gz#environment") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# necessary imports
from operator import add
import time
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F

# sentiment analysis libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### Define sentiment analysis function

In [3]:
def analyze_sentiment(text):
    """Determines the sentiment of a given text. 
    Returns 0 if text sentiment is neither positive nor negative.
    Returns 1 if text sentiment is mainly positive
    Returns -1 if text sentiment is mainly negative. """
    try:
        sia = SentimentIntensityAnalyzer()
    except:
        nltk.download('vader_lexicon')
        sia = SentimentIntensityAnalyzer()
    try:
        scores = sia.polarity_scores(text)
    except: 
        pass
    pos = scores['pos']
    neg = scores['neg']
    if pos == neg:
        return 0
    elif pos > neg:
        return 1
    else:
        return -1

In [4]:
# check function
print(analyze_sentiment('I love you!'))
print(analyze_sentiment('I hate you..'))

1
-1


### Check how the sentiment of a post relates to it's score:
- Do posts with positive sentiment have a higher average score than negative posts?

In [5]:
def apply_sentiment_analysis(path):
    # read in data from path
    df = spark_session.read.json(path)
    # select columns
    df = df.select('body', 'score').cache()
    # instantiate user defined function
    analyze_sentiment_fct = udf(analyze_sentiment, IntegerType())
    # apply 'analyze_sentiment' on 'body and save result in column 'sentiment'
    df = df.withColumn('sentiment', analyze_sentiment_fct(df.body))
    
    # only consider posts with score larger than 10
    df = df.filter(df.score > 15)
    
    # posts with positive sentiment
    df_neg = df.filter(df.sentiment == -1)
    
    # posts with negative sentiment
    df_pos = df.filter(df.sentiment == 1)
    
    # num of posts with positive / negative sentiment
    num_neg = df_neg.count()
    num_pos = df_pos.count()
    print('num_neg ', num_neg)
    print('num_pos ', num_pos)
    
    # total score
    sum_score_neg = df_neg.groupBy().agg(F.sum('score')).collect()
    sum_score_pos = df_pos.groupBy().agg(F.sum('score')).collect()
    
    print('sum_score_neg ', sum_score_neg[0][0])
    print('sum_score_pos ', sum_score_pos[0][0])

    # average score per post for negative / positive sentiment
    avg_neg = abs(sum_score_neg[0][0])/num_neg
    avg_pos = abs(sum_score_pos[0][0])/num_pos
     
    return avg_neg, avg_pos

### Measure runtime

In [6]:
file_1_path = "hdfs://192.168.2.207:9000/input/RC_2008-07"
file_2_path = "hdfs://192.168.2.207:9000/input/RC_2009-05"
file_3_path = "hdfs://192.168.2.207:9000/input/RC_2010-11"
#file_4_path = "hdfs://192.168.2.207:9000/input/RC_2011-08"

#files = [file_1_path, file_2_path, file_3_path, file_4_path]
files = [file_1_path, file_2_path, file_3_path]

for filepath in files:
    start_time = time.time()
    avg_neg, avg_pos = apply_sentiment_analysis(filepath)
    print('average score negative sentiment: ', avg_neg)
    print('average score pos sentiment: ', avg_pos)    
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

KeyboardInterrupt: 

In [ ]:
spark_session.stop()